#Preparacion de Entorno
Defino variables de entorno y contecto Google drive

In [ ]:
GENERATE_DATASET = True
IMPORT_NER = True
SEED = 223

In [ ]:
!pip install -qq faiss-gpu
!pip install -qq virtualenv
!pip install -qq selenium
!pip install -qq requests
!pip install -qq urllib3
!pip install -qq bs4
!pip install -qq PyMuPDF
!pip install -qq pytesseract
!sudo apt install -qq tesseract-ocr
!sudo apt install -qq libtesseract-dev

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 92.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-

In [ ]:
from google.colab import drive
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModel, pipeline, DistilBertTokenizer, DistilBertForSequenceClassification
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
import faiss
import os
import requests
import re
import sys
import string
import csv
import io
import fitz  # PyMuPDF
import pytesseract
from PIL import Image

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
drive.mount('/content/drive')

# Change the working directory to the project folder
%cd "/content/drive/"

# Set a random seed for reproducibility
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

Mounted at /content/drive
/content/drive




#Procesamiento de Texto



In [ ]:
pdf_folder = "/content/drive/MyDrive/ejecucion_metodologia/libro/"

pdfs = os.listdir(pdf_folder)

# Iterate through each PDF file in the folder
for pdf in pdfs:
    # List to hold text from each page
    text_pages = []

    text_pages.append(pdf)

    text_lines = []

    text_lines.append(pdf)

    pdf_path = pdf_folder + pdf

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)

    # Iterate through each page in the PDF
    for page_num in range(pdf_document.page_count):
        # Get the page
        page = pdf_document.load_page(page_num)

        # Render the page to an image
        pix = page.get_pixmap(dpi=200)

        # Convert the image to a PIL Image object
        img = Image.open(io.BytesIO(pix.tobytes()))

        # Convert to grayscale and apply binary thresholding
        img = img.convert('L')
        img = img.point(lambda x: 0 if x < 140 else 255, '1')

        # Perform OCR using pytesseract
        text = pytesseract.image_to_string(img, lang='eng')

        text_lines.append(text)

        text_pages.append(' '.join(text.split('\n')))

    # Combine text from all pages
    lines_full_text = '\n'.join(text_lines)
    pages_full_text = '\n'.join(text_pages)

    # Optionally, save the text to a file
    name_file = pdf.split('.')[0]

    output_folder_lines = '/content/drive/MyDrive/ejecucion_de_metodologia/lines/'
    output_folder_pages = '/content/drive/MyDrive//ejecucion_de_metodologia/pages/'
    os.makedirs(output_folder_lines, exist_ok=True)
    os.makedirs(output_folder_pages, exist_ok=True)

    with open(os.path.join(output_folder_lines, f'lines_{name_file}.txt'), 'w', encoding='utf-8') as f:
        f.write(lines_full_text)

    with open(os.path.join(output_folder_pages, f'pages_{name_file}.txt'), 'w', encoding='utf-8') as f:
        f.write(pages_full_text)

    print(f"Text extracted from {pdf}")

Text extracted from BOOK Adam Sharr Modern Architecture.pdf


#Identificacion de segmento

In [ ]:
model_path = 'lucasdefino/text-segmentation'

tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.to('cuda')

unique_labels = ['Paragraph', 'Title', 'Caption']
def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to('cuda')
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    predicted_label = unique_labels[prediction]
    return predicted_label

# Definimos las carpetas de Google Drive
input_folder = '/content/drive/MyDrive/ejecucion_metodologia/lines'
output_folder = '/content/drive/MyDrive/ejecucion_metodologia/csvs_text_segmentation'
os.makedirs(output_folder, exist_ok=True)

# Procesamos cada libro
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_file_path = os.path.join(input_folder, filename)
        output_filename = f"TS {filename.replace('BOOK', '').replace('.txt', '').strip()}.csv"
        output_file_path = os.path.join(output_folder, output_filename)

        with open(input_file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        classified_results = []
        for line in lines:
            stripped_line = line.strip()
            if stripped_line:
                label = classify_text(stripped_line)
                classified_results.append({'text': stripped_line, 'label': label})

        # Guardamos los resultados a un CSV por libro
        df = pd.DataFrame(classified_results)
        df.to_csv(output_file_path, index=False)

        print(f"Processed {filename} and saved as {output_filename}")

print("All files processed and saved.")

Processed lines_BOOK Adam Sharr Modern Architecture.txt and saved as TS lines_ Adam Sharr Modern Architecture.csv
All files processed and saved.


#NER + RAG + Scraping WIKI

##RAG Definition

###Flan T5 model

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
flan_t5_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large").to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

###Generation intructions

In [ ]:
def generate_answer(question: str, input_dict, max_length: int = 2046) -> tuple[str, str]:
    prompt = "Answer the following question about architectural buildings, using your expertise in historical and contemporary architecture and considering all provided contexts. Be concise and answer only the question.\n\n"
    prompt += f"Question: {question}\n"

    # Conseguir y concatenar los contextos
    contexts = retrieve_contexts(question, input_dict)
    for i, ctx in enumerate(contexts, 1):
        prompt += f"Context {i}: {ctx}\n\n"

    prompt += "The final answer is:\n"

    inputs = t5_tokenizer.encode_plus(
        prompt,
        return_tensors="pt",
        max_length=max_length,
        truncation=True
    ).to('cuda')

    outputs = flan_t5_model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        num_beams=5,
        early_stopping=True
    )

    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    del inputs, outputs, contexts, prompt

    return '', answer

In [ ]:
def verify_arch(arch: str, input_dict, max_length: int = 2046) -> tuple[str, str]:
    prompt = '''Classify the input as a valid architectural building based on the given contexts. Remember, only standalone architectural structures qualify; locations, streets, and people's names do not. Respond with 'Yes' or 'No' only. Here are some examples:
                Input: Casa Vicens
                Output: Yes

                Input: Bois de Boulogne
                Output: No

                Input: Maison Dom-Ino
                Output: Yes

                Input: Rue de la Harpe
                Output: No

                Input: Gallerie des Machines
                Output: Yes

                Input: Le Corbusier
                Output: No
                '''

    prompt += f"Input: {arch}\n"

    contexts = retrieve_contexts(prompt, input_dict)
    for i, ctx in enumerate(contexts[:2], 1):
        prompt += f"Context {i}: {ctx}\n\n"

    inputs = t5_tokenizer.encode_plus(
        prompt,
        return_tensors="pt",
        max_length=max_length,
        truncation=False
    ).to('cuda')

    outputs = flan_t5_model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        num_beams=5,
        early_stopping=True
    )

    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    del inputs, outputs, prompt

    return '', answer

In [ ]:
def infer_tipologia(arch: str, input_dict, max_length: int = 2046) -> tuple[str, str]:
    prompt = '''Given the following definition:
    A building's functional type is a designation that classifies buildings by their originally intended use or purpose at the time of construction (e.g., house, sanatorium, office). While the functional type is sometimes part of a building's name (e.g., "Farnsworth House" implies "house"), it may also be absent or misleading (e.g., "Seagram Building" does not directly indicate a function, and "Les Arcades du Lac" is residential, not commercial). Additionally, buildings may have overlapping designations (e.g., "house", "residence", or "villa").

    Using the provided contexts, infer and output the building's functional type as a single word or short phrase.'''

    contexts = retrieve_contexts(prompt, input_dict)

    prompt += f"Building: {arch}\n"
    for i, ctx in enumerate(contexts[:2]):
        prompt += f"Context {i}: {ctx}\n\n"
    prompt += "Output:\n"

    inputs = t5_tokenizer.encode_plus(
        prompt,
        return_tensors="pt",
        max_length=max_length,
        truncation=False
    ).to('cuda')

    outputs = flan_t5_model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        num_beams=5,
        early_stopping=False
    )

    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    del inputs, outputs, contexts, prompt

    return '', answer

def classify_tipologia(arch: str, input_dict, max_length: int = 2046) -> tuple[str, str]:
    prompt = '''
    Analyze the following architecture with your expertise in historical and contemporary architecture, considering the provided context. Classify it into one of the functional types listed below, and state your classification at the end.
    Functional Types: Abbey - Customs Office - Altar - Asylum / Orphanage - Studio - Auditorium - University Auditorium - Lecture Hall - Town Hall or Municipality - Bank (branch or headquarters) - Public Baths - Library - Winery - Stock Exchange - Bowling Alley - Cabins - Bathhouse - Retreat House - Rest Home - Casino / Gambling House - Cemetery - Server Center - Telephone Center - Civic Center - Cultural or Exhibition Center - Convention Center - Exhibition Center - Innovation Center - Research Center - Logistics Center - Congress or Convention Palace - Sports Center - Cinema - Club - Clubhouse - Professional Public School - Holiday Camp - Summer Camp - Congress, Parliament, or Assembly - Municipal Council - Clinic - Convent - Post Office - Court of Justice - Crematorium - General Warehouse - Train Depot - Specialized Education - Embassy or Consulate - Facility - Primary School - Secondary School - Fire Station - Subway Station - Police Station - Service Station - Railway Station - Indoor Sports Stadium - Baseball Stadium - Football Stadium - Olympic Stadium - Radio Broadcasting Studios - Factory - Commercial Arcade - Gym - Farm / Stable - Hangar - Hybrid - Hippodrome - Hospital / Clinic / Dispensary - Hostel - Hotel - Vertical Garden - Church - Printing House - Greenhouse / Botanical Garden - Courts - Kindergarten - Laboratory - Master Plan - Slaughterhouse - Media Library - Memorial - Wholesale / Central Market - Retail Market - Mosque - Ministry or Government Department - Observation Tower - Monastery - Motel - Museum - Observatory - Government Administrative Offices - General Offices - Exhibition Pavilion - Meeting Pavilion - Government Palace - Pantheon - Beach Resort - Parking Garage - Parliament or Assembly - Park - Water Park - Penitentiary - Pool / Swimming Facility - Power Plant - Recycling Plant - Bridge - Recycling Facility - Shelter - Renovation - Government Residence and Palace - Restaurant / Bar / Coffee Shop - University Services (for students, etc.) - Shopping Mall - Silo - Synagogue - Supermarket - Theater - Airport Terminal - Bus Terminal - Passenger Port Terminal - Retail Store / Showroom - Department Store - Single-Brand Store - University or College - Velodrome - Collective Housing - Single-Family Housing - Housing for Pensioners - University Housing - Zoo    '''

    # contexts = retrieve_contexts(prompt, input_dict)

    prompt += f"Architecture: {arch}\n"
    # for i, ctx in enumerate(contexts[:2]):
    #     prompt += f"Context {i}: {ctx}\n\n"
    prompt += "Output:\n"

    inputs = t5_tokenizer.encode_plus(
        prompt,
        return_tensors="pt",
        max_length=max_length,
        truncation=False
    ).to('cuda')

    outputs = flan_t5_model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        num_beams=5,
        early_stopping=False
    )

    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

    del inputs, outputs, prompt

    return '', answer

### Faiss index

In [ ]:
# Load model from HuggingFace Hub
st_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
st_model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens').to('cuda')

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Encoding contexts
def encode_contexts(contexts: list[str], model: AutoModel, batch_size: int = 8) -> np.ndarray:
    model.eval()
    encoded_contexts = []
    with torch.no_grad():
        for i in range(0, len(contexts), batch_size):
            batch = contexts[i:i+batch_size]
            inputs = st_tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=512).to('cuda')
            outputs = model(**inputs)
            encoded_batch = mean_pooling(outputs, inputs['attention_mask']).cpu().numpy()
            encoded_contexts.append(encoded_batch)
    return np.vstack(encoded_contexts)

# Build FAISS index
def build_faiss_index(contexts: list[str], model: AutoModel, batch_size: int = 8) -> faiss.IndexFlatL2:
    context_embeddings = encode_contexts(contexts, model, batch_size)
    dimension = context_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(context_embeddings)
    return index

# Code for semantic search
def semantic_search(query: str, index: faiss.IndexFlatL2, model: AutoModel, k: int = 3) -> list[int]:
    model.eval()
    with torch.no_grad():
        inputs = st_tokenizer(query, return_tensors='pt', padding=True, truncation=True, max_length=512).to('cuda')
        outputs = model(**inputs)
        query_embedding = mean_pooling(outputs, inputs['attention_mask']).cpu().numpy().reshape(1, -1)

    D, I = index.search(query_embedding, k)

    # Return indices of top k retrieved contexts
    return I[0].tolist()

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

##Fuctions

###Useful functions

In [ ]:
titles = ['Mrs.', 'Mr.', 'Dr.', 'Ms.']
uppers = [chr(i) + '.' for i in range(65, 91)]
lowers = [ ' ' + chr(i) + '.' for i in range(97, 122)]
numbers = [str(i) + '.' for i in range(0, 10)]

def remove_titles(text):
    mytext = text

    for title in titles:
        mytext = mytext.replace(title, title[:-1])
    for upper in uppers:
        mytext = mytext.replace(upper, upper[:-1])
    for number in numbers:
        mytext = mytext.replace(number, number[:-1]+',')
    for lower in lowers:
        mytext = mytext.replace(lower, lower[:-1])

    return mytext

def prepare_data(filename):
    txt = open(filename, 'r')
    txt = txt.read().replace('\n', ' ')
    txt = txt.replace('_', ' ')
    txt = txt.replace('—', '-')
    txt = txt.replace('–', '-')
    txt = txt.replace('“', '"')
    txt = txt.replace('”', '"')
    txt = txt.replace('’', "'")
    txt = txt.replace('‘', "'")
    txt = txt.replace('…', ' ')
    txt = txt.replace('...', ' ')
    txt = txt.replace('|', 'I')
    txt = txt.replace('+', ' ')
    txt = txt.replace('/', ' ')
    txt = txt.replace('(', ' ')
    txt = txt.replace(')', ' ')
    txt = txt.replace(':', ' ')
    txt = txt.replace(';', ' ')

    pattern = r"\[\d+\]"
    txt = re.sub(pattern, "", txt)
    txt = re.sub(' +', ' ', txt)
    txt = remove_titles(txt)
    data = [sentence.strip() for sentence in txt.split('.') if sentence.strip() != '']

    return data

def create_context(arch: str, content: str):
    """
    Retrieves all the pages where arch appeared in content
    """
    pages = []
    split_content = content.split('\n')
    for i in range(len(split_content)):
        page = split_content[i]
        previous_page = split_content[i-1] if i > 0 else None
        if arch in page:
          pages.append(page.strip())
    return pages

def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

def remove_stopwords(text: str):
  stop_words = set(stopwords.words('english'))
  words = text.split()
  filtered_words = [word for word in words if word.lower() not in stop_words]
  return ' '.join(filtered_words)

def count_and_position_correction(arch: str, txt: str):
    text = txt.lower()
    arch = arch.lower()

    matches = list(re.finditer(re.escape(arch), text))
    positions = [(m.start(), m.end()) for m in matches]
    count = max(len(matches), 1)

    return count, positions

def retrieve_contexts(question, input_dict):
    contexts = input_dict['contexts']
    index = input_dict['book_index']

    wiki_contexts = input_dict['wiki_contexts']
    wiki_index = input_dict['wiki_index']

    small_wiki_contexts = []
    small_contexts = []
    if wiki_index is not None:
      return [wiki_contexts[i] for i in semantic_search(question, wiki_index, st_model, 2)]
    elif wiki_contexts != []:
      return wiki_contexts
    elif index is not None:
      return [contexts[i] for i in semantic_search(question, index, st_model, 2)]
    elif contexts != []:
      return contexts
    else:
      return []

###Scrapping Functions

In [ ]:
def generate_url(architecture_name):
    formatted_name = architecture_name.replace(" ", "_").title()

    lowercase_words = [
        # French
        "De", "Des", "Du", "La", "Le", "Les", "Aux", "Et", "En", "Sur", "À",
        # English
        "Of", "The", "And", "A", "An", "In", "On", "For", "At",
        # Spanish
        "De", "Del", "La", "El", "Los", "Las", "Y", "En", "Con", "Por", "Para", "Un", "Una",
        # Italian
        "Di", "Della", "Del", "Lo", "Il", "I", "Gli", "Le", "E", "Nel", "Con", "Per", "Tra", "Fra"
    ]

    # Split the name into words and correct the casing for articles/prepositions
    formatted_name_parts = formatted_name.split("_")
    formatted_name_parts = [word if word not in lowercase_words else word.lower() for word in formatted_name_parts]

    # Join the words back with underscores
    formatted_name = "_".join(formatted_name_parts)

    url = f"https://en.wikipedia.org/wiki/{formatted_name}"
    return url

def scrape_architect(architect_name, architecture_name):
    url = generate_url(architect_name)

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Check if the page is a disambiguation or not found
        disambig_phrases = [
            "Other reasons this message may be displayed:",
            f"may refer to:"
        ]

        # If any disambiguation or not-found phrase appears, skip saving
        if any(phrase in str(soup) for phrase in disambig_phrases):
          return []

        # Extract paragraphs
        paragraphs = soup.find_all('p')

        pattern = r"\[\d+\]"

        contexts = [re.sub(pattern, "", p.get_text().replace("\n", '')) for p in paragraphs if p.get_text().replace("\n", '') != '']

        return [context for context in contexts if architecture_name.lower() in context.lower()]

    except Exception as e:
        return []

# Function to scrape Wikipedia page
def scrape_architecture(architecture_name, index, contexts):
    url = generate_url(architecture_name)

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Check if the page is a disambiguation or not found
        disambig_phrases = [
            "Other reasons this message may be displayed:",
            f"may refer to:"
        ]

        # If any disambiguation or not-found phrase appears, skip saving
        if any(phrase in str(soup) for phrase in disambig_phrases):
          question = f"Who was/were the architect/s behind the {architecture_name}?"

          input_dict = {
              'contexts': contexts,
              'book_index': index,
              'wiki_contexts': [],
              'wiki_index': None
          }

          prompt, answer = generate_answer(question, input_dict)

          return scrape_architect(answer, architecture_name)

        # Extract paragraphs
        paragraphs = soup.find_all('p')

        pattern = r"\[\d+\]"

        return [re.sub(pattern, "", p.get_text().replace("\n", '')) for p in paragraphs if p.get_text().replace("\n", '') != '']

    except Exception as e:
        question = f"Who was/were the architect/s behind the {architecture_name}?"

        input_dict = {
            'contexts': contexts,
            'book_index': index,
            'wiki_contexts': [],
            'wiki_index': None
        }

        prompt, answer = generate_answer(question, input_dict)

        return scrape_architect(answer, architecture_name)


###Result functions

In [ ]:
def text_segmentation_results(architecture: str, book):
  # Initialize a dictionary to store counts for each architecture
  counts = {'qty_title': 0, 'qty_paragraph': 0, 'qty_caption': 0}

  for _, row in book.ts_df.iterrows():
    text = row['text'].lower()
    label = row['label']
    if architecture.lower() in text:
      if label == 'Title':
        counts['qty_title'] += 1
      elif label == 'Paragraph':
        counts['qty_paragraph'] += 1
      elif label == 'Caption':
        counts['qty_caption'] += 1

  return counts

# Question for CSV final csv

column_questions = {
    'author/s': "Who was/were the architect/s behind the {arch}?",
    'city': "In which city is the {arch} located?",
    'country': "In which country is the {arch} situated?",
    'start_year': "In which year did the construction of the {arch} begin?",
    'end_year': "In which year was the construction of the {arch} completed?"
}

def generate_questions(archs, topics):
    questions = []
    for arch in archs:
        for topic, question_template in topics.items():
            question = question_template.format(arch=arch)
            questions.append(question)
    return questions

In [ ]:
def generate_result_line(input_dict: dict, book):
  output_dict = {}

  output_dict['architecture'] = input_dict['arch']

  output_dict['is_architecture'] = "Yes"

  input_text = input_dict['arch']

  questions = generate_questions([input_text], column_questions)

  for k, question in enumerate(questions):

    if k == 2:
      question = f"In which country is {output_dict['city']} situated?"

    prompt, answer = generate_answer(question, input_dict)

    column = list(column_questions.keys())[k]

    output_dict[column] = answer

  ts_counts = text_segmentation_results(input_dict['arch'], book)
  output_dict.update(ts_counts)
  output_dict['qty_unassigned'] = input_dict['count'] - sum(ts_counts.values())
  output_dict['qty'] = input_dict['count']
  output_dict['certainty_score'] = sum(input_dict["scores"]) / len(input_dict["scores"])

  prompt, answer = infer_tipologia(input_dict['arch'], input_dict)
  output_dict['fuctional_type_inferred'] = answer

  prompt, answer = classify_tipologia(input_dict['arch'], input_dict)
  output_dict['fuctional_type_classified'] = answer

  output_dict['uses_wikipedia'] = "True" if input_dict['wiki_contexts'] != [] else "False"

  return output_dict

def get_results(dataset, book):
  results = []
  for i in tqdm(range(len(dataset))):
    arch = dataset[i]['arch']
    prompt, answer = verify_arch(arch, dataset[i])
    if "yes" in answer.lower():
      results.append(generate_result_line(dataset[i], book))
    else:
      results.append({'architecture': arch, "is_architecture": "No", 'author/s': '', 'city': '', 'country': '', 'start_year': '', 'end_year': '', 'qty_title': '', 'qty_paragraph': '', 'qty_caption': '', 'qty_unassigned': '', 'qty': '', 'certainty_score': '', 'fuctional_type_inferred': '', 'fuctional_type_classified': '', 'uses_wikipedia': ''})

    torch.cuda.empty_cache()
  return results

##Datasets Definition

In [ ]:
if IMPORT_NER:
  ner = pipeline(
      'token-classification',
      model="lucasdefino/architecture-NER",
      tokenizer="lucasdefino/architecture-NER",
      aggregation_strategy='simple',
      device=0
  )

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
class BookDataset(Dataset):
    def __init__(self, book_file_path, txt_seg_file_path):
        self.txt: str = read_file(book_file_path)
        self.data: list[str] = prepare_data(book_file_path)
        self.ts_df = pd.read_csv(txt_seg_file_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def identify_and_process_architectures(
    book: BookDataset, model, batch_size: int, confidence: float = 0.9, min_len: int = 7
):
    results = model(book.data, batch_size=batch_size)

    processed_archs = {}

    for i, result in enumerate(tqdm(results, desc="Identifying and processing architectures")):
        for ent in result:
            if ent['entity_group'] == 'ARCH' and len(ent['word']) > min_len and ent['score'] > confidence:
                if '#' in ent['word']:
                    while ent['start'] > 0 and book.data[i][ent['start'] - 1] != ' ':
                        ent['start'] -= 1
                    ent['word'] = book.data[i][ent['start']:ent['end']]

                word = remove_stopwords(ent['word'])

                if word in processed_archs:
                    processed_archs[word]['scores'].append(ent['score'])
                else:
                    count, positions = count_and_position_correction(word, book.txt)

                    contexts = create_context(word, book.txt)

                    book_index = (
                        build_faiss_index(contexts, st_model) if len(contexts) > 0 else None
                    )

                    wiki_contexts = scrape_architecture(word, book_index, contexts)
                    wiki_index = (
                        build_faiss_index(wiki_contexts, st_model)
                        if len(wiki_contexts) > 0
                        else None
                    )

                    processed_archs[word] = {
                        'arch': word,
                        'positions': positions,
                        'scores': [ent['score']],
                        'contexts': contexts,
                        'wiki_contexts': wiki_contexts,
                        'count': count,
                        'book_index': book_index,
                        'wiki_index': wiki_index,
                    }

    return processed_archs

class ArchitectureDataset(Dataset):
    def __init__(self, processed_archs: dict):
        """
        processed_archs: A dictionary containing processed architecture entity information.
        Each key is an architecture word, and its value is a dictionary with 'arch', 'positions', 'scores', 'contexts', 'count', 'index'.
        """
        self.archs = list(processed_archs.values())

    def __len__(self):
        return len(self.archs)

    def __getitem__(self, idx):
        return self.archs[idx]

In [ ]:
if GENERATE_DATASET:
  pages_folder = "/content/drive/MyDrive/ejecucion_metodologia/pages/"
  csvs_folder = "/content/drive/MyDrive/ejecucion_metodologia/csvs_text_segmentation/"

  file_name = next(os.walk(pages_folder))[2][0]
  ts_file_name = next(os.walk(csvs_folder))[2][0]

  file_path = os.path.join(pages_folder, file_name)
  ts_file_path = os.path.join(csvs_folder, ts_file_name)

  book = BookDataset(file_path, ts_file_path)
  processed_archs = identify_and_process_architectures(
      book=book,
      model=ner,
      batch_size=16,
      confidence=0.8,
      min_len=7,
  )

  dataset = ArchitectureDataset(processed_archs)

Identifying and processing architectures: 100%|██████████| 1630/1630 [03:00<00:00,  9.01it/s]


#Results Generation

##Results extraction

In [ ]:
# ADVERTENCIA: por limitaciones de tiempo de GPU puede suceder que un libro muy extenso no llegue a terminar de ejecutarse.
#              En estos casos se debe separar el dataset en baches y ejecutarlos en distintas sesiones.

results = get_results(dataset, book)

100%|██████████| 138/138 [25:00<00:00, 10.87s/it]


In [ ]:
fieldnames = ['architecture', 'is_architecture', 'author/s', 'city', 'country', 'start_year', 'end_year', 'qty_title', 'qty_paragraph', 'qty_caption', 'qty_unassigned', 'qty', 'certainty_score', 'fuctional_type_inferred', 'fuctional_type_classified', 'uses_wikipedia']

csv_buffer = io.StringIO()

writer = csv.DictWriter(csv_buffer, fieldnames=fieldnames)
writer.writeheader()
writer.writerows(results)

csv_buffer.seek(0)

with open('/content/drive/MyDrive/ejecucion_metodologia/resultado/base_arch_book.csv', mode='w', encoding='utf-8') as file:
    file.write(csv_buffer.getvalue())

print("CSV file created successfully!")


CSV file created successfully!
